In [2]:
from fastai.text.all import *

In [3]:
mkdir 'data'

In [4]:
!cp -r ../input/poemsdataset/forms/ data

In [5]:
path = Path('data/forms/')
Path.BASE_PATH = path
path.ls()

In [6]:
files = get_text_files(path)
files

In [7]:
txt = files[0].open().read(); txt[:75]

In [8]:
get_poem = partial(get_text_files)

dls_lm = DataBlock(
    blocks=TextBlock.from_folder(path, is_lm=True),
    get_items=get_poem, splitter=RandomSplitter(0.1)
).dataloaders(path, path=path, bs=128, seq_len=10)

In [9]:
dls_lm.show_batch(max_n=2)

In [10]:
learn = language_model_learner(
    dls_lm, AWD_LSTM, drop_mult=0.3, 
    metrics=[accuracy, Perplexity()]).to_fp16()

In [11]:
learn.fit_one_cycle(1, 2e-2)

In [12]:
learn.unfreeze()
learn.fit_one_cycle(10, 2e-3)

# test generation

In [17]:
TEXT = "in the heaven"
N_WORDS = 10
N_SENTENCES = 2
preds = [learn.predict(TEXT, N_WORDS, temperature=0.75) 
         for _ in range(N_SENTENCES)]

In [18]:
print("\n".join(preds))

In [19]:
TEXT = "I see the light"
N_WORDS = 10
N_SENTENCES = 2
preds = [learn.predict(TEXT, N_WORDS, temperature=0.75) 
         for _ in range(N_SENTENCES)]

In [20]:
print("\n".join(preds))